In [4]:
import os
import copy
import numpy as np
import pandas as pd
import time
import datetime
import math
import matplotlib.pyplot as plt
import uniswappy


In [3]:
import sys
import subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", "/Users/ashertobinchodos/Documents/uniswappy_thrack"])


Obtaining file:///Users/ashertobinchodos/Documents/uniswappy_thrack


ERROR: file:///Users/ashertobinchodos/Documents/uniswappy_thrack does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


CalledProcessError: Command '['/opt/anaconda3/envs/unsiwappy/bin/python', '-m', 'pip', 'install', '-e', '/Users/ashertobinchodos/Documents/uniswappy_thrack']' returned non-zero exit status 1.

In [8]:
import sys
print(sys.executable)


/opt/anaconda3/envs/unsiwappy/bin/python


In [4]:
df = pd.read_csv('weth_dai_events.csv')   

init_mint = df[df.event == 'mint'].iloc[0]
dai_amount = int(init_mint['amount0'])
weth_amount = int(init_mint['amount1'])
init_hash = init_mint['evt_tx_hash']

df = df[df.evt_tx_hash != init_hash]

FileNotFoundError: [Errno 2] No such file or directory: 'weth_dai_events.csv'

In [ ]:
fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

dai = ERC20("DAI", "0x111")
weth = ERC20("WETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = weth, tkn1 = dai, symbol="LP",
                                   address="0x011", version = 'V3',
                                   tick_spacing = tick_spacing,
                                   fee = fee)

factory = UniswapFactory("SYS pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.precision = 'GWEI'
user_nm = 'user0'

Join().apply(lp, user_nm, dai_amount, weth_amount, lwr_tick, upr_tick)

lp.get_price(weth)

NameError: name 'UniV3Utils' is not defined

In [ ]:
df[df.liquidity_provider == '0x98744b988bb5cfd7f8fab9556b744fc009a70ac8']

,event,source,evt_tx_hash,evt_index,evt_block_time,evt_block_number,amount_lp_token,amount0,amount1,tokenId,first_recipient,liquidity_provider,tickLower,tickUpper,pool_liquidity,trader,sqrtPriceX96,tick
107,mint,pool-and-nfpm,0xc762a8dc7ce45ea04447b7b147b0c02e477785c7fcf7...,166,2021-05-05 18:57:01.000 UTC,12375982,84966387866432909482621,4999999999999999998980,2896992847310586746,239.0,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,-81360.0,-81300.0,NaN,0x,NaN,NaN
115,burn,pool-and-nfpm,0x180248db584ef3659f228aa5d65d0fdd6e0c46cf2b3f...,90,2021-05-05 19:12:46.000 UTC,12376045,-84966387866432909482621,-14870209017648091725660,0,239.0,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,-81360.0,-81300.0,NaN,0x,NaN,NaN
128,mint,pool-and-nfpm,0xea78c75b937760c6f96dc742c5bbf1906d81ced8531e...,55,2021-05-05 19:32:40.000 UTC,12376147,7431475210743243877524,21677328817142363787670,3000000000000000000,382.0,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,-81900.0,-80400.0,NaN,0x,NaN,NaN
2043,burn,pool-and-nfpm,0xc696f4905cf877c1ce383008166bbb3f556ca9e934b0...,220,2021-05-09 15:33:31.000 UTC,12400981,-7431475210743243877524,-32231321676793547962579,0,382.0,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,0x98744b988bb5cfd7f8fab9556b744fc009a70ac8,-81900.0,-80400.0,NaN,0x,NaN,NaN


In [ ]:
for i, row in df[:1000].iterrows():
    match row.event:
        case 'mint':
            lp.mint(
                row.liquidity_provider,
                int(row.tickLower),
                int(row.tickUpper),
                int(row.amount_lp_token)
            )
        case 'burn':
           print(row)
           lp.burn(
                row.liquidity_provider,
                int(row.tickLower),
                int(row.tickUpper),
                abs(int(row.amount_lp_token))
            )

        # case 'swap':
        #     if row.amount0 > 0:
        #         Swap().apply(lp, user_nm, weth, dai, int(row.amount0))
        #     else:
        #         Swap().apply(lp, user_nm, dai,_


event                                                              burn
source                                                    pool-and-nfpm
evt_tx_hash           0x6a3ba2478ce7bbba2abba8006cd43c9647c6ddb6b5a2...
evt_index                                                           153
evt_block_time                              2021-05-05 00:29:05.000 UTC
evt_block_number                                               12370988
amount_lp_token                                    -4648242676661787499
amount0                                                               0
amount1                                              -49999999999999999
tokenId                                                            39.0
first_recipient              0x0459b3fbf7c1840ee03a63ca4aa95de48322322e
liquidity_provider           0x0459b3fbf7c1840ee03a63ca4aa95de48322322e
tickLower                                                     -109260.0
tickUpper                                                      -

AssertionError: UniswapV3: INSUFFICIENT_SUBTRACT_AMOUNT